# 익명처리 - 수형자정보

### 1. 데이터 로드

In [1]:
import random
import pandas as pd
import os
import numpy as np
import datetime
import time
from scipy.stats import mode

In [2]:
file_path = 'dataset.xlsx'

# 엑셀 파일 열기 (모든 시트 정보를 로딩)
xls = pd.ExcelFile(file_path)

# 시트 이름 리스트 확인 (필요 시)
print(xls.sheet_names)  # ['Sheet1', 'Sheet2', ...]

['수형자 정보', '수형자 월별 활동 정보', '수형자 면회기록']


In [3]:
df_info = pd.read_excel(xls, sheet_name=xls.sheet_names[0])
#df_act = pd.read_excel(xls, sheet_name=xls.sheet_names[1])
#df_visit = pd.read_excel(xls, sheet_name=xls.sheet_names[2])

In [4]:
info_annonymized_data = df_info.copy()

### 1. 식별자 칼럼 삭제

In [5]:
remove_columns = ['주민등록번호', '성명']
info_annonymized_data[remove_columns] = None

### 2. DATETIME 칼럼 범주화 

In [6]:
info_annonymized_data['생년월일'] = pd.to_datetime(info_annonymized_data['생년월일']).dt.year
info_annonymized_data['입소일'] = pd.to_datetime(info_annonymized_data['입소일']).dt.year
info_annonymized_data['가석방 예정 일자'] = pd.to_datetime(info_annonymized_data['가석방 예정 일자']).dt.to_period('Q') #분기별

In [7]:
info_annonymized_data['가석방 예정 일자'].head()

0    NaT
1    NaT
2    NaT
3    NaT
4    NaT
Name: 가석방 예정 일자, dtype: period[Q-DEC]

#### 2-1. 생년월일 => 연령대 처럼 추가 범주화 진행 -> 추가 1920~1950 로컬 일반화

In [8]:
info_annonymized_data['생년월일'] = info_annonymized_data['생년월일'].apply(
    lambda x: f'[{(x // 10) * 10}, {(x // 10) * 10 + 10})'
)

# 결과 확인
print("=== 구간별 레코드 수 ===")
interval_counts = info_annonymized_data['생년월일'].value_counts().sort_index()
print(interval_counts)

print("\n=== 샘플 데이터 확인 ===")
print(info_annonymized_data[['성별', '생년월일']].head(10))

print("\n=== 각 구간의 원본 연도 범위 확인 ===")
for interval in sorted(info_annonymized_data['생년월일'].unique()):
    years_in_interval = info_annonymized_data[info_annonymized_data['생년월일'] == interval]['생년월일']
    min_year = years_in_interval.min()
    max_year = years_in_interval.max()
    count = len(years_in_interval)
    print(f"{interval}: {min_year}~{max_year} ({count}건)")

=== 구간별 레코드 수 ===
생년월일
[1920, 1930)       80
[1930, 1940)      973
[1940, 1950)     2925
[1950, 1960)     8578
[1960, 1970)    18789
[1970, 1980)    22724
[1980, 1990)    19746
[1990, 2000)    18281
[2000, 2010)     7904
Name: count, dtype: int64

=== 샘플 데이터 확인 ===
  성별          생년월일
0  F  [2000, 2010)
1  M  [2000, 2010)
2  F  [2000, 2010)
3  M  [2000, 2010)
4  M  [2000, 2010)
5  M  [2000, 2010)
6  M  [2000, 2010)
7  M  [2000, 2010)
8  F  [2000, 2010)
9  M  [2000, 2010)

=== 각 구간의 원본 연도 범위 확인 ===
[1920, 1930): [1920, 1930)~[1920, 1930) (80건)
[1930, 1940): [1930, 1940)~[1930, 1940) (973건)
[1940, 1950): [1940, 1950)~[1940, 1950) (2925건)
[1950, 1960): [1950, 1960)~[1950, 1960) (8578건)
[1960, 1970): [1960, 1970)~[1960, 1970) (18789건)
[1970, 1980): [1970, 1980)~[1970, 1980) (22724건)
[1980, 1990): [1980, 1990)~[1980, 1990) (19746건)
[1990, 2000): [1990, 2000)~[1990, 2000) (18281건)
[2000, 2010): [2000, 2010)~[2000, 2010) (7904건)


In [9]:
info_annonymized_data['생년월일'] = info_annonymized_data['생년월일'].apply(
    lambda x: '[1920, 1960)' if x in ['[1920, 1930)', '[1930, 1940)', '[1940, 1950)', '[1950, 1960)'] else x
)

print("\n=== 각 구간의 원본 연도 범위 확인 ===")
for interval in sorted(info_annonymized_data['생년월일'].unique()):
    years_in_interval = info_annonymized_data[info_annonymized_data['생년월일'] == interval]['생년월일']
    min_year = years_in_interval.min()
    max_year = years_in_interval.max()
    count = len(years_in_interval)
    print(f"{interval}: {min_year}~{max_year} ({count}건)")


=== 각 구간의 원본 연도 범위 확인 ===
[1920, 1960): [1920, 1960)~[1920, 1960) (12556건)
[1960, 1970): [1960, 1970)~[1960, 1970) (18789건)
[1970, 1980): [1970, 1980)~[1970, 1980) (22724건)
[1980, 1990): [1980, 1990)~[1980, 1990) (19746건)
[1990, 2000): [1990, 2000)~[1990, 2000) (18281건)
[2000, 2010): [2000, 2010)~[2000, 2010) (7904건)


#### 2-2. 입소일 추가 범주화 (로컬일반화 2020년 기준)

In [10]:
def map_admission_year_simple(year):
    """연도 → 조정된 구간 매핑"""
    if year >= 2020:
        return '[2020, 2025)'
    else:
        return '[1980, 2020)'

# 적용
info_annonymized_data['입소일'] = info_annonymized_data['입소일'].apply(map_admission_year_simple)

print("\n=== 각 구간의 원본 연도 범위 확인 ===")
for interval in sorted(info_annonymized_data['입소일'].unique()):
    years_in_interval = info_annonymized_data[info_annonymized_data['입소일'] == interval]['입소일']
    min_year = years_in_interval.min()
    max_year = years_in_interval.max()
    count = len(years_in_interval)
    print(f"{interval}: {min_year}~{max_year} ({count}건)")


=== 각 구간의 원본 연도 범위 확인 ===
[1980, 2020): [1980, 2020)~[1980, 2020) (9112건)
[2020, 2025): [2020, 2025)~[2020, 2025) (90888건)


### 3. 주소 - 광역 단위로 범주화

In [11]:
address_list = ['주소']
for x in address_list:
    info_annonymized_data[x] = info_annonymized_data[x].apply(lambda x: x.split()[0] if isinstance(x, str) else None)

In [12]:
info_annonymized_data['주소'].value_counts()

주소
서울특별시    42595
부산광역시    15089
인천광역시    13209
대구광역시    10827
대전광역시     6729
광주광역시     6526
울산광역시     5025
Name: count, dtype: int64

### 4. 범죄유형 로컬일반화

In [13]:
# 출력 제한 해제
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 전체 범죄유형 빈도 출력
value_counts = info_annonymized_data['범죄유형'].value_counts()
print("=== 범죄유형 전체 빈도표 ===")
print(value_counts)

print(f"\n총 범죄유형 개수: {len(value_counts)}개")
print(f"총 레코드 수: {value_counts.sum():,}개")

=== 범죄유형 전체 빈도표 ===
범죄유형
사기/횡령    24696
기타       17315
강제추행     10062
과실범       8479
마약류       7974
폭력        7513
절도        6353
살인미수      4268
강간        3700
강도        2533
특수강도      1655
살인        1587
교통        1557
유사강간      1270
기타강간       869
방화         103
선거          66
Name: count, dtype: int64

총 범죄유형 개수: 17개
총 레코드 수: 100,000개


In [14]:
# 범죄유형 로컬일반화 (선거, 방화 => 기타 && 기타강간, 유사강간 => 강간)
crime_recategorization = {
    '선거': '기타',           # 66개 → 기타로 통합
    '방화': '기타',           # 103개 → 기타로 통합  
    '기타강간': '강간',        # 869개 → 강간으로 통합
    '유사강간': '강간'         # 1,270개 → 강간으로 통합
}

info_annonymized_data['범죄유형'] = info_annonymized_data['범죄유형'].replace(crime_recategorization)

In [15]:
# 전체 범죄유형 빈도 출력
value_counts = info_annonymized_data['범죄유형'].value_counts()
print("=== 범죄유형 전체 빈도표 ===")
print(value_counts)

print(f"\n총 범죄유형 개수: {len(value_counts)}개")
print(f"총 레코드 수: {value_counts.sum():,}개")

=== 범죄유형 전체 빈도표 ===
범죄유형
사기/횡령    24696
기타       17484
강제추행     10062
과실범       8479
마약류       7974
폭력        7513
절도        6353
강간        5839
살인미수      4268
강도        2533
특수강도      1655
살인        1587
교통        1557
Name: count, dtype: int64

총 범죄유형 개수: 13개
총 레코드 수: 100,000개


### 5. 교도소 범주화

In [16]:
info_annonymized_data['교도소'].value_counts()

교도소
대전교도소        7397
대구교도소        6176
안양교도소        6105
여주교도소        5638
목포교도소        4955
포항교도소        4668
순천교도소        4668
경북북부제3교도소    4560
천안개방교도소      4489
창원교도소        4130
부산교도소        4094
의정부교도소       4058
서울남부교도소      3950
전주교도소        3806
광주교도소        3735
청주교도소        2693
춘천교도소        2585
원주교도소        2514
김천소년교도소      2442
안동교도소        2334
청주여자교도소      2190
공주교도소        1975
해남교도소        1797
영월교도소        1795
상주교도소        1760
속초교도소        1716
정읍교도소        1580
홍성교도소        1185
강릉교도소        1005
Name: count, dtype: int64

In [17]:
def categorize_prison_to_region(prison_name):
    """실제 데이터의 29개 교도소를 5개 권역으로 분류"""
    
    prison_regions = {
        # 중부권 (서울, 경기, 충청권, 대전)
        '중부권': [
            '안양교도소', '여주교도소', '의정부교도소', '서울남부교도소',
            '천안개방교도소', '청주교도소', '청주여자교도소', '공주교도소',
            '홍성교도소', '대전교도소'
        ],
        
        # 남부권 (부산, 경남, 전남, 광주)
        '남부권': [
            '부산교도소', '창원교도소', '목포교도소', '순천교도소',
            '해남교도소', '광주교도소'
        ],
        
        # 동부권 (대구, 경북)
        '동부권': [
            '대구교도소', '포항교도소', '경북북부제3교도소',
            '안동교도소', '상주교도소', '김천소년교도소'
        ],
        
        # 서부권 (전북)
        '서부권': [
            '전주교도소', '정읍교도소'
        ],
        
        # 북부권 (강원)
        '북부권': [
            '춘천교도소', '원주교도소', '영월교도소',
            '속초교도소', '강릉교도소'
        ]
    }
    
    # 교도소명으로 권역 찾기
    for region, prisons in prison_regions.items():
        if prison_name in prisons:
            return region
    
    # 매핑되지 않은 경우
    return '기타권역'

# 테스트
test_prisons = ['대전교도소', '포항교도소', '목포교도소']
for prison in test_prisons:
    print(f"{prison} → {categorize_prison_to_region(prison)}")

# 적용 예시
def apply_prison_regionalization(df):
    """데이터프레임에 교도소 권역화 적용"""
    df['교도소'] = df['교도소'].apply(categorize_prison_to_region)
    return df

대전교도소 → 중부권
포항교도소 → 동부권
목포교도소 → 남부권


In [18]:
info_annonymized_data = apply_prison_regionalization(info_annonymized_data)

# 결과 확인
print("=== 권역별 레코드 수 ===")
region_counts = info_annonymized_data['교도소'].value_counts()
print(region_counts)

print("\n=== 각 권역별 교도소 목록 ===")
for region in ['중부권', '남부권', '동부권', '서부권', '북부권']:
    prisons_in_region = info_annonymized_data[info_annonymized_data['교도소'] == region]['교도소'].value_counts()
    print(f"\n{region}:")
    for prison, count in prisons_in_region.items():
        print(f"  {prison}: {count}건")

=== 권역별 레코드 수 ===
교도소
중부권    39680
남부권    23379
동부권    21940
북부권     9615
서부권     5386
Name: count, dtype: int64

=== 각 권역별 교도소 목록 ===

중부권:
  중부권: 39680건

남부권:
  남부권: 23379건

동부권:
  동부권: 21940건

서부권:
  서부권: 5386건

북부권:
  북부권: 9615건


### 6. 자격증 범주화

In [19]:
def categorize_license(license):
    """실제 데이터의 29개 교도소를 5개 권역으로 분류"""
    
    licenses = {
        # 기술직
        '기술직': [
            '건설', '기계', '전기전자', '재료', '정보통신'
        ],
        
        # 서비스직
        '서비스직': [
            '음식서비스', '이용', '청소'
        ],
        
        # 농림어업/식품가공
        '농림어업/식품가공': [
            '농림어업', '식품가공'
        ],
        
        # 디자인/공예/패션
        '디자인/공예/패션': [
            '디자인', '섬유의복', '인쇄, 목재, 가구, 공예'
        ],
        
        # 없음
        '없음': [
            '없음'
        ]
    }
    
    for license_name, categories in licenses.items():
        if license in categories:
            return license_name
    
    # 매핑되지 않은 경우
    return '기타'

# 적용 예시
def apply_license_categorization(df):
    df['자격증종류'] = df['자격증종류'].apply(categorize_license)
    return df

In [20]:
info_annonymized_data = apply_license_categorization(info_annonymized_data)

# 결과 확인
print("=== 자격증별 레코드 수 ===")
region_counts = info_annonymized_data['자격증종류'].value_counts()
print(region_counts)

=== 자격증별 레코드 수 ===
자격증종류
없음           90936
기술직           6389
서비스직          1479
농림어업/식품가공      660
디자인/공예/패션      536
Name: count, dtype: int64


### 수치형 데이터 붙여넣기

In [21]:
info_annonymized_data.head()

,일련번호,수형자ID,주민등록번호,성명,성별,생년월일,주소,입소일,형량_개월,범죄유형,형기감소일,가석방예정여부,가석방 예정 일자,일일 평균 작업시간,일일 평균 교육시간,일일 평균 운동시간,징계율,포상율,자격증취득여부,자격증종류,교도소,재범여부,재범횟수
0,1,1862638,None,None,F,"[2000, 2010)",서울특별시,"[2020, 2025)",35.0,기타,138,N,NaT,4.73,1.68,0.67,0.0,0.179,N,없음,중부권,N,0
1,2,1761657,None,None,M,"[2000, 2010)",서울특별시,"[2020, 2025)",21.0,폭력,18,N,NaT,4.80,1.49,0.50,0.0,0.714,N,없음,남부권,N,0
2,3,1006722,None,None,F,"[2000, 2010)",부산광역시,"[2020, 2025)",33.0,기타,0,N,NaT,4.44,0.57,0.57,0.0,0.000,N,없음,동부권,Y,0
3,4,1154809,None,None,M,"[2000, 2010)",인천광역시,"[2020, 2025)",44.0,폭력,197,N,NaT,5.04,0.78,0.77,0.0,0.534,N,없음,북부권,N,0
4,5,1619398,None,None,M,"[2000, 2010)",서울특별시,"[2020, 2025)",39.0,마약류,224,N,NaT,5.12,0.97,0.59,0.0,0.196,N,없음,중부권,Y,2


In [22]:
file_path = 'output_1.xlsx'

# 엑셀 파일 열기 (모든 시트 정보를 로딩)
output = pd.read_excel(file_path)

info_annonymized_data = info_annonymized_data.set_index('수형자ID')
output_1_data = output.set_index('수형자ID')

# 덮어쓸 컬럼명
target_cols = ['형량_개월', '형기감소일']

# 기존 값에 덮어쓰기
for col in target_cols:
    if col in info_annonymized_data.columns:
        info_annonymized_data.loc[output_1_data.index, col] = output_1_data[col]
    else:
        # 만약 info_annonymized_data에 해당 칼럼이 없다면 새로 추가
        info_annonymized_data[col] = pd.NA
        info_annonymized_data.loc[output_1_data.index, col] = output_1_data[col]

# 인덱스 원복
info_annonymized_data = info_annonymized_data.reset_index()

In [23]:
info_annonymized_data.head()

,수형자ID,일련번호,주민등록번호,성명,성별,생년월일,주소,입소일,형량_개월,범죄유형,형기감소일,가석방예정여부,가석방 예정 일자,일일 평균 작업시간,일일 평균 교육시간,일일 평균 운동시간,징계율,포상율,자격증취득여부,자격증종류,교도소,재범여부,재범횟수
0,1862638,1,None,None,F,"[2000, 2010)",서울특별시,"[2020, 2025)",35.0,기타,138,N,NaT,4.73,1.68,0.67,0.0,0.179,N,없음,중부권,N,0
1,1761657,2,None,None,M,"[2000, 2010)",서울특별시,"[2020, 2025)",21.0,폭력,18,N,NaT,4.80,1.49,0.50,0.0,0.714,N,없음,남부권,N,0
2,1006722,3,None,None,F,"[2000, 2010)",부산광역시,"[2020, 2025)",33.0,기타,0,N,NaT,4.44,0.57,0.57,0.0,0.000,N,없음,동부권,Y,0
3,1154809,4,None,None,M,"[2000, 2010)",인천광역시,"[2020, 2025)",44.0,폭력,197,N,NaT,5.04,0.78,0.77,0.0,0.534,N,없음,북부권,N,0
4,1619398,5,None,None,M,"[2000, 2010)",서울특별시,"[2020, 2025)",39.0,마약류,224,N,NaT,5.12,0.97,0.59,0.0,0.196,N,없음,중부권,Y,2


### K-익명성 (K=8) 체크

In [24]:
def check_k_anonymity(df, quasi_identifiers, k):
    """
    df: 데이터프레임
    quasi_identifiers: 리스트, 준식별자 컬럼 목록
    k: 만족시켜야 할 최소 그룹 크기
    """
    # 준식별자 기준으로 그룹핑 후 크기 계산
    group_sizes = df.groupby(quasi_identifiers).size().reset_index(name='count')

    # k보다 작은 그룹이 있는지 확인
    violating_groups = group_sizes[group_sizes['count'] < k]

    is_k_anonymous = violating_groups.empty

    return is_k_anonymous, violating_groups

In [25]:
quasi_identifiers = ['성별', '생년월일', '주소', '입소일', '가석방 예정 일자', '자격증종류', '교도소']
is_k_anonymous, violating_groups = check_k_anonymity(info_annonymized_data, quasi_identifiers, 8)
print(f"[K=8 익명성 만족 여부]:{is_k_anonymous}")
print(f"[총 위반 그룹 수]: {len(violating_groups)}")
print(f"[그 중 그룹 크기별 개수 분포]\n{violating_groups['count'].value_counts().sort_index()}")

[K=8 익명성 만족 여부]:False
[총 위반 그룹 수]: 1174
[그 중 그룹 크기별 개수 분포]
count
1    629
2    218
3    116
4     76
5     57
6     45
7     33
Name: count, dtype: int64


# 면회기록 데이터 로드

In [26]:
file_path_visit = 'output_3.xlsx'

# 엑셀 파일 열기 (모든 시트 정보를 로딩)
xls_visit = pd.ExcelFile(file_path_visit)

df_visit = pd.read_excel(xls_visit, sheet_name=xls_visit.sheet_names[0])

In [27]:
df_act = pd.read_excel(xls, sheet_name=xls.sheet_names[1])

### 수형자ID 해시처리

In [28]:
import hashlib
import os
import base64

# 16바이트 랜덤 salt 생성 (Base64 인코딩해서 저장 가능)
salt = base64.b64encode(os.urandom(16)).decode('utf-8')

def hash_identifier(identifier, salt='salt'):
    string_to_hash = f"{identifier}_{salt}"
    return hashlib.sha256(string_to_hash.encode()).hexdigest()

In [29]:
info_annonymized_data['수형자ID'] = info_annonymized_data['수형자ID'].apply(lambda x: hash_identifier(x, salt))
df_act['수형자ID'] = df_act['수형자ID'].apply(lambda x: hash_identifier(x, salt))
df_visit['수형자ID'] = df_visit['수형자ID'].apply(lambda x: hash_identifier(x, salt))

In [30]:
df_act.head()

,수형자ID,2023년 1월 평균 작업시간,2023년 2월 평균 작업시간,2023년 3월 평균 작업시간,2023년 4월 평균 작업시간,2023년 5월 평균 작업시간,2023년 6월 평균 작업시간,2023년 7월 평균 작업시간,2023년 8월 평균 작업시간,2023년 9월 평균 작업시간,2023년 10월 평균 작업시간,2023년 11월 평균 작업시간,2023년 12월 평균 작업시간,2023년 1월 평균 교육시간,2023년 2월 평균 교육시간,2023년 3월 평균 교육시간,2023년 4월 평균 교육시간,2023년 5월 평균 교육시간,2023년 6월 평균 교육시간,2023년 7월 평균 교육시간,2023년 8월 평균 교육시간,2023년 9월 평균 교육시간,2023년 10월 평균 교육시간,2023년 11월 평균 교육시간,2023년 12월 평균 교육시간,2023년 1월 평균 운동시간,2023년 2월 평균 운동시간,2023년 3월 평균 운동시간,2023년 4월 평균 운동시간,2023년 5월 평균 운동시간,2023년 6월 평균 운동시간,2023년 7월 평균 운동시간,2023년 8월 평균 운동시간,2023년 9월 평균 운동시간,2023년 10월 평균 운동시간,2023년 11월 평균 운동시간,2023년 12월 평균 운동시간
0,295f9dd63264688fde7cc276f8aa874045424466cae6ec...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,116.3,77.9,123.3,67.5,108.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.6,18.0,20.4,17.8,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.7,16.3,18.7,15.0,13.1
1,9bbadfc8f083255fe13b15a947c49afc4bbadac04d5927...,100.6,86.6,115.0,121.5,87.1,106.9,117.7,121.4,97.8,101.0,90.7,104.9,30.5,23.0,21.9,16.4,16.5,18.3,36.1,40.4,35.9,15.6,34.0,38.2,12.8,21.6,18.3,14.6,14.7,17.7,17.8,18.1,17.7,18.4,18.2,11.2
2,8589bdbf2495393e0315e17d212eae82cd32529c80da89...,98.9,77.9,75.8,115.4,125.8,100.7,73.5,121.2,96.6,96.9,118.4,73.7,37.1,11.5,31.1,33.1,11.6,23.6,15.7,39.2,37.1,21.3,43.7,30.1,20.0,18.4,21.7,12.6,18.1,16.3,14.0,13.3,20.2,18.4,13.8,20.2
3,09d9be837c8b5ed4b970046af6f59f8324ac8927ac78d3...,130.5,121.4,113.4,121.7,131.1,97.8,110.8,118.4,97.7,105.8,74.5,111.9,18.4,33.3,38.4,18.0,15.3,41.4,11.5,39.8,30.3,40.8,20.6,12.5,21.6,12.8,18.6,21.2,21.0,11.6,14.4,13.0,19.5,12.4,18.5,14.1
4,c6f6e9b8d76664f9e31a2a91b8250c2159e0a969cfbcd4...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.8,76.9,98.0,67.2,126.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.9,28.2,15.7,35.1,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.8,21.2,19.9,16.5,12.6


In [31]:
df_visit.head()

,수형자ID,면회일자,면회유형,방문자관계
0,7a7b7c7d80e4f5f152b898cd56884616132ee8ee9933b4...,2023-08,일반,형제자매
1,1945f6c30ad482f957400d23a96a00d4c48713b32ac349...,2023-05,일반,형제자매
2,9e5a7289c04e70924915490e18867c4cb46318126c5bb9...,2023-07,일반,기타
3,ac791487b83cf6531c4cb8117e695ac73c0b463440796a...,2023-06,일반,형제자매
4,254439bd469d69a00c03e5f1719a16196956f77eef26c2...,2023-10,특별,법조인


In [32]:
info_annonymized_data.head()

,수형자ID,일련번호,주민등록번호,성명,성별,생년월일,주소,입소일,형량_개월,범죄유형,형기감소일,가석방예정여부,가석방 예정 일자,일일 평균 작업시간,일일 평균 교육시간,일일 평균 운동시간,징계율,포상율,자격증취득여부,자격증종류,교도소,재범여부,재범횟수
0,10990e30e7324bb4dffe147cb091273e23db21f8d350dc...,1,None,None,F,"[2000, 2010)",서울특별시,"[2020, 2025)",35.0,기타,138,N,NaT,4.73,1.68,0.67,0.0,0.179,N,없음,중부권,N,0
1,ce644d7412f2ce086051f1779b60b69d0cfad816531d1f...,2,None,None,M,"[2000, 2010)",서울특별시,"[2020, 2025)",21.0,폭력,18,N,NaT,4.80,1.49,0.50,0.0,0.714,N,없음,남부권,N,0
2,ab0c8126e54a89e464a5371f39edf408499d27c821a82f...,3,None,None,F,"[2000, 2010)",부산광역시,"[2020, 2025)",33.0,기타,0,N,NaT,4.44,0.57,0.57,0.0,0.000,N,없음,동부권,Y,0
3,fe42205d17da1855d297370d4674adef4b051cce7cbb88...,4,None,None,M,"[2000, 2010)",인천광역시,"[2020, 2025)",44.0,폭력,197,N,NaT,5.04,0.78,0.77,0.0,0.534,N,없음,북부권,N,0
4,9ddea2172173aca427d225034e3b4aee23c3524867d0d2...,5,None,None,M,"[2000, 2010)",서울특별시,"[2020, 2025)",39.0,마약류,224,N,NaT,5.12,0.97,0.59,0.0,0.196,N,없음,중부권,Y,2


In [33]:
df_act.to_csv('./예선_데이터셋_(학생트랙)_(흑석동물주먹)_sheet2.csv', encoding="utf-8", index=False)
df_visit.to_csv('./예선_데이터셋_(학생트랙)_(흑석동물주먹)_sheet3.csv', encoding="utf-8", index=False)


# K-익명성 만족하지 않는 것 Suppresion

In [36]:
if not is_k_anonymous:
    print("\n[k-익명성 위반 레코드 마스킹 시작...]")
    
    # '일련번호'를 제외한 모든 컬럼 목록
    columns_to_mask = [col for col in info_annonymized_data.columns if col != '일련번호']
    
    masked_count = 0  # 마스킹된 레코드 수 카운트
    
    # 위반 그룹별로 처리
    for idx, row in violating_groups.iterrows():
        # 해당 그룹의 조건 만들기
        condition = pd.Series(True, index=info_annonymized_data.index)
        
        for col in quasi_identifiers:
            condition &= (info_annonymized_data[col] == row[col])
        
        # 해당 조건에 맞는 행들의 개수 확인
        matching_rows = info_annonymized_data[condition]
        masked_count += len(matching_rows)
        
        # '일련번호'를 제외한 모든 컬럼을 None으로 설정
        info_annonymized_data.loc[condition, columns_to_mask] = None
        
        print(f"  그룹 {idx+1}: {len(matching_rows)}개 레코드 마스킹 완료")
    
    print(f"\n[마스킹 완료] 총 {masked_count}개 레코드가 마스킹되었습니다.")
    
    # 마스킹 후 다시 k-익명성 검사 (검증용)
    is_k_anonymous_after, violating_groups_after = check_k_anonymity(info_annonymized_data, quasi_identifiers, 8)
    print(f"[마스킹 후 K=8 익명성 만족 여부]: {is_k_anonymous_after}")
    print(f"[마스킹 후 위반 그룹 수]: {len(violating_groups_after)}")

else:
    print("k-익명성을 이미 만족하므로 마스킹이 필요하지 않습니다.")


[k-익명성 위반 레코드 마스킹 시작...]
  그룹 1: 0개 레코드 마스킹 완료
  그룹 2: 0개 레코드 마스킹 완료
  그룹 3: 0개 레코드 마스킹 완료
  그룹 4: 0개 레코드 마스킹 완료
  그룹 5: 0개 레코드 마스킹 완료
  그룹 6: 0개 레코드 마스킹 완료
  그룹 7: 0개 레코드 마스킹 완료
  그룹 8: 0개 레코드 마스킹 완료
  그룹 9: 0개 레코드 마스킹 완료
  그룹 10: 0개 레코드 마스킹 완료
  그룹 11: 0개 레코드 마스킹 완료
  그룹 12: 0개 레코드 마스킹 완료
  그룹 13: 0개 레코드 마스킹 완료
  그룹 14: 0개 레코드 마스킹 완료
  그룹 15: 0개 레코드 마스킹 완료
  그룹 16: 0개 레코드 마스킹 완료
  그룹 17: 0개 레코드 마스킹 완료
  그룹 18: 0개 레코드 마스킹 완료
  그룹 19: 0개 레코드 마스킹 완료
  그룹 20: 0개 레코드 마스킹 완료
  그룹 21: 0개 레코드 마스킹 완료
  그룹 22: 0개 레코드 마스킹 완료
  그룹 23: 0개 레코드 마스킹 완료
  그룹 24: 0개 레코드 마스킹 완료
  그룹 25: 0개 레코드 마스킹 완료
  그룹 26: 0개 레코드 마스킹 완료
  그룹 27: 0개 레코드 마스킹 완료
  그룹 28: 0개 레코드 마스킹 완료
  그룹 29: 0개 레코드 마스킹 완료
  그룹 30: 0개 레코드 마스킹 완료
  그룹 31: 0개 레코드 마스킹 완료
  그룹 32: 0개 레코드 마스킹 완료
  그룹 33: 0개 레코드 마스킹 완료
  그룹 34: 0개 레코드 마스킹 완료
  그룹 35: 0개 레코드 마스킹 완료
  그룹 36: 0개 레코드 마스킹 완료
  그룹 38: 0개 레코드 마스킹 완료
  그룹 39: 0개 레코드 마스킹 완료
  그룹 40: 0개 레코드 마스킹 완료
  그룹 41: 0개 레코드 마스킹 완료
  그룹 42: 0개 레코드 마스킹 완료
  그룹 43: 0개 레코드 마스킹 완료
  그룹 44: 0개 레코드 마

In [37]:
info_annonymized_data.head()

,수형자ID,일련번호,주민등록번호,성명,성별,생년월일,주소,입소일,형량_개월,범죄유형,형기감소일,가석방예정여부,가석방 예정 일자,일일 평균 작업시간,일일 평균 교육시간,일일 평균 운동시간,징계율,포상율,자격증취득여부,자격증종류,교도소,재범여부,재범횟수
0,10990e30e7324bb4dffe147cb091273e23db21f8d350dc...,1,None,None,F,"[2000, 2010)",서울특별시,"[2020, 2025)",35.0,기타,138.0,N,NaT,4.73,1.68,0.67,0.0,0.179,N,없음,중부권,N,0.0
1,ce644d7412f2ce086051f1779b60b69d0cfad816531d1f...,2,None,None,M,"[2000, 2010)",서울특별시,"[2020, 2025)",21.0,폭력,18.0,N,NaT,4.80,1.49,0.50,0.0,0.714,N,없음,남부권,N,0.0
2,ab0c8126e54a89e464a5371f39edf408499d27c821a82f...,3,None,None,F,"[2000, 2010)",부산광역시,"[2020, 2025)",33.0,기타,0.0,N,NaT,4.44,0.57,0.57,0.0,0.000,N,없음,동부권,Y,0.0
3,fe42205d17da1855d297370d4674adef4b051cce7cbb88...,4,None,None,M,"[2000, 2010)",인천광역시,"[2020, 2025)",44.0,폭력,197.0,N,NaT,5.04,0.78,0.77,0.0,0.534,N,없음,북부권,N,0.0
4,9ddea2172173aca427d225034e3b4aee23c3524867d0d2...,5,None,None,M,"[2000, 2010)",서울특별시,"[2020, 2025)",39.0,마약류,224.0,N,NaT,5.12,0.97,0.59,0.0,0.196,N,없음,중부권,Y,2.0


In [38]:
info_annonymized_data.to_csv('./예선_데이터셋_(학생트랙)_(흑석동물주먹)_sheet1.csv', encoding="utf-8", index=False)

# 평가 지표

In [39]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

class DeIdentificationMetrics:
    """
    등급표에 정의된 6개 유용성 지표 안전 검증 구현
    """

    def __init__(self, original: pd.DataFrame, deidentified: pd.DataFrame, id_col: str = 'id'):
        self.orig = original.copy()
        self.deid = deidentified.copy()
        self.id_col = id_col

        # 공통 id만 유지 정렬
        common = set(self.orig[id_col]) & set(self.deid[id_col])
        self.orig = self.orig[self.orig[id_col].isin(common)].sort_values(id_col)
        self.deid = self.deid[self.deid[id_col].isin(common)].sort_values(id_col)

    # U1 코사인 유사도 (수치형 평균)
    def u1_cs(self, sens_cols: list) -> float:
        vals = []
        for col in sens_cols:
            if col not in self.orig or col not in self.deid:
                continue
            # 수치형 컬럼만 처리
            if not pd.api.types.is_numeric_dtype(self.orig[col]) or not pd.api.types.is_numeric_dtype(self.deid[col]):
                continue
            x = self.orig[col].to_numpy(dtype=float).reshape(1, -1)
            y = self.deid[col].to_numpy(dtype=float).reshape(1, -1)
            if x.shape[1] < 2:
                continue
            x, y = np.nan_to_num(x), np.nan_to_num(y)
            try:
                v = cosine_similarity(x, y)[0][0]
                if not np.isnan(v):
                    vals.append(v)
            except Exception:
                continue
        return float(np.mean(vals)) if vals else np.nan

    # U2 평균 상관계수 차 (2개 미만이면 np.nan)
    def u2_mc(self, sens_cols: list) -> float:
        # 수치형 컬럼만 선택
        cols = [
            c for c in sens_cols
            if c in self.orig and c in self.deid
            and pd.api.types.is_numeric_dtype(self.orig[c])
            and pd.api.types.is_numeric_dtype(self.deid[c])
        ]
        if len(cols) < 2:
            return np.nan

        df_o = self.orig[cols].astype(float).dropna()
        df_d = self.deid[cols].astype(float).dropna()
        if len(df_o) < 2 or len(df_d) < 2:
            return np.nan

        corr_o = df_o.corr(method='pearson')
        corr_d = df_d.corr(method='pearson')
        diffs = []
        for i in range(len(cols)):
            for j in range(i + 1, len(cols)):
                co = corr_o.iloc[i, j]
                cd = corr_d.iloc[i, j]
                if not (np.isnan(co) or np.isnan(cd)):
                    diffs.append(abs(co - cd))
        return float(np.mean(diffs)) if diffs else np.nan

    # U3 카테고리 트리 구조 일반화 평균 차이
    @staticmethod
    def _level(code: str, sep='>') -> list:
        return str(code).split(sep)

    def u3_mgd_ca(self, cat_cols: list, sep='>') -> float:
        vals = 0
        cnt = 0
        for col in cat_cols:
            if col not in self.orig or col not in self.deid:
                continue
            merged = pd.merge(
                self.orig[[self.id_col, col]],
                self.deid[[self.id_col, col]],
                on=self.id_col,
                suffixes=('_o', '_d')
            )
            for _, row in merged.iterrows():
                p1 = self._level(row[f'{col}_o'], sep)
                p2 = self._level(row[f'{col}_d'], sep)
                depth = min(len(p1), len(p2))
                idx = next((i for i in range(depth) if p1[i] != p2[i]), depth)
                dist = (len(p1) - idx) + (len(p2) - idx)
                norm = dist / (len(p1) + len(p2)) if (len(p1) + len(p2)) > 0 else 0
                vals += norm
                cnt += 1
        return float(vals / cnt) if cnt > 0 else np.nan

    # U4 동질집합군 평균 분산 (QI, sens_numeric 혹은 sens_categorical 모두 지원)
    def u4_md_ecm(self, qi_cols: list, sens_cols: list) -> float:
        # QI 컬럼
        qi = [c for c in qi_cols if c != self.id_col and c in self.deid]
        if not qi:
            return np.nan

        # 감쇠변수: 수치형은 분산, 범주형은 집단 내 비율 불균형 검사(예시로 엔트로피 사용)
        variances = 0.0
        counts = 0

        grouped = self.deid.groupby(qi)
        for _, grp in grouped:
            for col in sens_cols:
                if col not in grp:
                    continue
                series = grp[col].dropna()
                if series.empty:
                    continue
                if pd.api.types.is_numeric_dtype(series):
                    if len(series) >= 2:
                        v = np.var(series.values, ddof=0)
                        variances += v
                        counts += 1
                else:
                    # 범주형: 엔트로피로 분산 대체
                    freqs = series.value_counts(normalize=True)
                    ent = -np.sum(freqs * np.log2(freqs + 1e-9))
                    variances += ent
                    counts += 1

        return float(variances / counts) if counts > 0 else np.nan

    # U5 EC 크기 계산 (QI 하나라도 없으면 nan)
    def u5_na_ecsm(self, qi_cols: list, k_val: int = 3) -> float:
        qi = [c for c in qi_cols if c != self.id_col and c in self.deid]
        if not qi or k_val <= 0:
            return np.nan
        ec_count = self.deid.groupby(qi).ngroups
        if ec_count == 0:
            return np.nan
        res = (len(self.deid) / ec_count) / k_val
        return float(res) if np.isfinite(res) else np.nan

    # U6 익명화 비율
    def u6_ar(self) -> float:
        if len(self.orig) == 0:
            return np.nan
        return float(100.0 * len(self.deid) / len(self.orig))

    def calculate_metrics(self, qi_cols: list, sens_cols: list, cat_cols: list, k_val: int = 3):
        return {
            'U1_CS'     : self.u1_cs(sens_cols),
            'U2_MC'     : self.u2_mc(sens_cols),
            'U3_MGD_CA' : self.u3_mgd_ca(cat_cols),
            'U4_MD_ECM' : self.u4_md_ecm(qi_cols, sens_cols),
            'U5_NA_ECSM': self.u5_na_ecsm(qi_cols, k_val),
            'U6_AR'     : self.u6_ar()
        }

In [40]:
def evaluate_metric(metric: str, value: float) -> str:
    if value is None or np.isnan(value) or not np.isfinite(value):
        return 'N/A'
    if metric == 'U1_CS':
        if value >= 0.95: return 'Excellent'
        if value >= 0.80: return 'Good'
        if value >= 0.50: return 'Fair'
        return 'Poor'
    if metric == 'U2_MC':
        if value < 0.05: return 'Excellent'
        if value < 0.20: return 'Good'
        if value < 0.50: return 'Fair'
        return 'Poor'
    if metric == 'U3_MGD_CA':
        if value < 0.05: return 'Excellent'
        if value < 0.15: return 'Good'
        if value < 0.30: return 'Fair'
        return 'Poor'
    if metric == 'U4_MD_ECM':
        if value < 0.10: return 'Excellent'
        if value < 1.00: return 'Good'
        if value < 5.00: return 'Fair'
        return 'Poor'
    if metric == 'U5_NA_ECSM':
        if value < 1.0: return 'Excellent'
        if value < 2.0: return 'Good'
        if value < 3.0: return 'Fair'
        return 'Poor'
    if metric == 'U6_AR':
        if value >= 95: return 'Excellent'
        if value >= 80: return 'Good'
        if value >= 50: return 'Fair'
        return 'Poor'
    return 'Unknown'

In [43]:
# 1. 데이터 준비
file_path = 'dataset.xlsx'
# 엑셀 파일 열기 (모든 시트 정보를 로딩)
xls = pd.ExcelFile(file_path)
# 두 번째 시트 불러오기 (인덱스 1은 두 번째 시트)
original_data = pd.read_excel(xls, sheet_name=xls.sheet_names[0])

# deidentified_data = pd.read_csv('./예선_데이터셋_(학생트랙)_(흑석동물주먹)_(완료).csv')
deidentified_data = info_annonymized_data
# 2. 평가 객체 생성
metrics = DeIdentificationMetrics(original_data, deidentified_data, id_col='일련번호')

# 3. 컬럼 정의
quasi_identifiers = [
        '성별', '생년월일', '주소', '입소일', '가석방 예정일자', '자격증종류', '교도소'
    ]
sensitive_columns = ['형량_개월', '범죄유형', '형기감소일', '가석방예정여부', '재범여부', '재범횟수']  # 민감속성
string_columns = ['주소', '입소일', '범죄유형', '가석방예정여부', '가석방 예정 일자', '자격증취득여부', '자격증종류', '교도소', '재범여부']  # 문자열 컬럼



In [44]:
scores = metrics.calculate_metrics(quasi_identifiers, sensitive_columns, string_columns, k_val=8)

# 5) 등급 부여
for name, val in scores.items():
    out = float(val) if val is not None and np.isfinite(val) and not np.isnan(val) else np.nan
    print(f'{name}: {out:.4f}' if not np.isnan(out) else f'{name}: N/A', f'({evaluate_metric(name, out)})')

U1_CS: 0.8921 (Good)
U2_MC: 0.0114 (Excellent)
U3_MGD_CA: 0.3652 (Poor)
U4_MD_ECM: 370.2039 (Poor)
U5_NA_ECSM: 6.4935 (Poor)
U6_AR: 100.0000 (Excellent)


In [45]:
info_annonymized_data[['일련번호', '수형자ID', '주민등록번호', '성명', '성별', '생년월일', '주소', '입소일', '형량_개월',
       '범죄유형', '형기감소일', '가석방예정여부', '가석방 예정 일자', '일일 평균 작업시간', '일일 평균 교육시간',
       '일일 평균 운동시간', '징계율', '포상율', '자격증취득여부', '자격증종류', '교도소', '재범여부', '재범횟수']].to_csv('change_1.csv', index=False, encoding='utf-8')